In [1]:
# Sklearn imports
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder
from sklearn.ensemble import RandomForestClassifier
import pandas as pd

# Tensorflow import
import tensorflow as tf
import os

C:\Users\diddy\Anaconda3\lib\site-packages\numpy\_distributor_init.py:32: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\diddy\Anaconda3\lib\site-packages\numpy\.libs\libopenblas.GK7GX5KEQ4F6UYO3P26ULGBQYHGQO7J4.gfortran-win_amd64.dll
C:\Users\diddy\Anaconda3\lib\site-packages\numpy\.libs\libopenblas.XWYDX2IKJW2NMTWSFYNGFUWKQU3LYTCZ.gfortran-win_amd64.dll
  stacklevel=1)
C:\Users\diddy\Anaconda3\lib\site-packages\pandas\compat\_optional.py:138: UserWarning: Pandas requires version '2.7.0' or newer of 'numexpr' (version '2.6.9' currently installed).
  warnings.warn(msg, UserWarning)


In [2]:
from model_interfaces import base_model
import data, dice
import model as dice_model

In [3]:
dataset = pd.read_csv('adultdataset.csv')

In [4]:
dataset.head()

,age,workclass,education,marital_status,occupation,race,gender,hours_per_week,income
0,28,Private,Bachelors,Single,White-Collar,White,Female,60,0
1,30,Self-Employed,Assoc,Married,Professional,White,Male,65,1
2,32,Private,Some-college,Married,White-Collar,White,Male,50,0
3,20,Private,Some-college,Single,Service,White,Female,35,0
4,41,Self-Employed,Some-college,Married,White-Collar,White,Male,50,0


In [5]:
d = data.Data(dataframe=dataset, continuous_features=['age', 'hours_per_week'], outcome_name='income')

# Split data into train and test
target = dataset["income"]
datasetX = dataset.drop("income", axis=1)
x_train, x_test, y_train, y_test = train_test_split(datasetX,
                                                    target,
                                                    test_size=0.2,
                                                    random_state=0,
                                                    stratify=target)

numerical = ["age", "hours_per_week"]
categorical = x_train.columns.difference(numerical)

#データの前処理
categorical_transformer = Pipeline(steps=[
    ('onehot', OneHotEncoder(handle_unknown='ignore'))])

transformations = ColumnTransformer(
    transformers=[
        ('cat', categorical_transformer, categorical)])

# Append classifier to preprocessing pipeline.
# Now we have a full prediction pipeline.
clf = Pipeline(steps=[('preprocessor', transformations),
                      ('classifier', RandomForestClassifier())])
model = clf.fit(x_train, y_train)

# Using sklearn backend
m = dice_model.Model(model=model, backend="sklearn")
exp = dice.Dice(d, m, method="random")

e1 = exp.generate_counterfactuals(x_train[0:1], total_CFs=2, desired_class="opposite")
e1.visualize_as_dataframe(show_only_changes=True)

### Explaining a Tensorflow model

In [6]:
# supress deprecation warnings from TF
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)

backend = 'TF'+tf.__version__[0]  # TF2
#ML_modelpath = helpers.get_adult_income_modelpath(backend=backend)
model_ext = '.h5' if 'TF' in backend else '.pth'
ML_modelpath = os.path.join('utils', 'sample_trained_models', 'adult'+model_ext)

# Step 2: dice_ml.Model
m = dice_model.Model(model_path=ML_modelpath, backend=backend, model_type='regressor')
# Step 3: initiate DiCE
exp = dice.Dice(d, m)

In [7]:
# query instance in the form of a dictionary or a dataframe; keys: feature name, values: feature value
query_instance = {'age': 22,
                  'workclass': 'Private',
                  'education': 'HS-grad',
                  'marital_status': 'Single',
                  'occupation': 'Service',
                  'race': 'White',
                  'gender': 'Female',
                  'hours_per_week': 45}

In [8]:
# generate counterfactuals
dice_exp = exp.generate_counterfactuals(query_instance, total_CFs=2,
                                       desired_range=None, desired_class="opposite",
                                       verbose=False)

regressor


AttributeError: 'DiceTensorFlow2' object has no attribute 'target_cf_class'

In [ ]:
# visualize the result, highlight only the changes
dice_exp.visualize_as_dataframe(show_only_changes=True)

Query instance (original outcome: 1)


,age,workclass,education,marital_status,occupation,race,gender,hours_per_week,income
0,34.0,Private,Doctorate,Married,Service,White,Female,45.0,0.926



Diverse Counterfactual set (new outcome: 0.0)


,age,workclass,education,marital_status,occupation,race,gender,hours_per_week,income
0,20.0,-,-,Single,-,-,-,-,0
1,-,-,School,-,-,-,-,-,0
